In [ ]:
#neurons_prev_spk_time = [0, 0]


#prev_spk_time =[0, 0]
#isi = np.zeros((1, 2))
##################################################################################################
#   if np.any(spiked_neurons_mask): # One or more neurons have spiked at the current timestep.
    
#     # Enter the following block if none of the neurons had spiked previously i.e. the current
#     # timestep's spiking is the first time spiking.
#     if not np.any(neurons_prev_spk_time): # np.any([0, 0]) -> False
#       # Get the timestep of its first spike.
#       neurons_prev_spk_time[:] = spiked_neurons_mask*int_t
#       # If more than one neuron have spiked, choose any (Winner Take All method based on first spike).
#       # (although np.argmax returns the index of the first highest element).
#       max_pool_mask[np.argmax(neurons_prev_spk_time)] = 1.0
#       print("Int t", int_t)
#       print("MaxPool Mask", max_pool_mask)
#       print("X", x)
#       print("neurons_prev_spk_time", neurons_prev_spk_time)
#       print("*"*80)
    
#     # One or more neurons had spiked previously, now some neurons (could be just one neuron) 
#     # have spiked for the first time or again in the current timestep.
#     else:
#       # Calculate the ISI, few elements in `neurons_isi` will be negative for those neurons 
#       # which had spiked earlier, but didn't spike in current timestep.
#       neurons_isi = spiked_neurons_mask*int_t - neurons_prev_spk_time
      
#       # Check if any neuron has spiked earlier, if the same neuron spiked again in this timestep
#       # then calculate its ISI and set the max_pool_mask to the minimum ISI.
#       previously_spiked_mask = np.logical_not(np.isin(neurons_prev_spk_time, 0))
#       same_neurons_spiked_mask = previously_spiked_mask & spiked_neurons_mask
#       # Mask the non-same neurons (i.e. null their contribution) and calculate the index of 
#       # the minimum ISI and set the max_pool_mask.
#       print("Int t => Any of the neurons spiked at time: ", int_t)
#       print("X", x)
#       print("neurons_prev_spk_time", neurons_prev_spk_time)
#       print("Neurons ISI:", neurons_isi)
#       if np.any(same_neurons_spiked_mask):
#         max_pool_mask[:] = np.zeros(num_x) # Re-initialize the max_pool_mask to all 0s.
#         max_pool_mask[np.argmin(
#             nma.masked_array(neurons_isi, np.logical_not(same_neurons_spiked_mask)))] = 1.0
#         print("same_neurons_spiked_mask", same_neurons_spiked_mask)
#         print("*"*80)
      
#       # Update `neurons_prev_spk_time`.
#       neurons_prev_spk_time[:] += np.where(neurons_isi > 0, neurons_isi, neurons_isi*0)
#       print("MaxPool Mask", max_pool_mask)
#       print("Updated neurons_prev_spk_time", neurons_prev_spk_time)
#       print("#"*80)
####################################################################################
        
###############################################################################      
#   if prev_spk_time[0] == 0 and x[0] == 1000.0:
#       prev_spk_time[0] = int_t
#       isi[0, 0] = int_t
#   elif x[0] == 1000.0:
#       print("ISI 1st Neuron: ", int_t - prev_spk_time[0])
#       isi[0, 0] = int_t - prev_spk_time[0]
#       prev_spk_time[0] = int_t
  
#   if prev_spk_time[1] == 0 and x[1] == 1000.0:
#       prev_spk_time[1] = int_t
#       isi[0, 1] = int_t
#   elif x[1] == 1000.0:
#       print("ISI 2nd Neuron: ", int_t - prev_spk_time[1])
#       isi[0, 1] = int_t - prev_spk_time[1]
#       prev_spk_time[1] = int_t
  
#   # If any of the element in `isi` is non-zero it evaluates to True, thus supports TTFS method as well.
#   print("ISI: %s" % isi)
#   max_val_idx = np.argmin(isi)
#   max_pool_mask = np.array([1 if max_val_idx == 0 else 0, 1 if max_val_idx == 1 else 0])
#   print("MP Mask: ", max_pool_mask)
#   return np.dot(max_pool_mask, x)
##############################################################################

In [1]:
import _init_paths
from utils.cnn_2d_utils import get_2d_cnn_model
from utils.nengo_dl_utils import get_nengo_dl_model
from configs.exp_configs import tf_exp_cfg, nengo_dl_cfg
from utils.consts.exp_consts import NEURONS_LAST_SPIKED_TS, NEURONS_LATEST_ISI, MAX_POOL_MASK, ISI_BASED_MP_PARAMS
from utils.base_utils.exp_utils import get_isi_based_max_pooling_params

In [2]:
model = get_2d_cnn_model((1, 28, 28), tf_exp_cfg)

In [3]:
model[0].summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1, 28, 28)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 16, 26, 26)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 13, 13)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 11, 11)        3480      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 5, 5)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 3, 3)          6944      
_________________________________________________________________
flatten (Flatten)            (None, 288)               0     

In [2]:
ndl_model, ndl_probes = get_nengo_dl_model((1, 28, 28), tf_exp_cfg, nengo_dl_cfg)

/home/rgaurav/miniconda3/envs/latest-nengo-tf/lib/python3.7/site-packages/nengo_dl/converter.py:326: UserWarning: Cannot convert max pooling layers to native Nengo objects; consider setting max_to_avg_pool=True to use average pooling instead. Falling back to TensorNode.
  % (error_msg + ". " if error_msg else "")
/home/rgaurav/miniconda3/envs/latest-nengo-tf/lib/python3.7/site-packages/nengo_dl/converter.py:588: UserWarning: Activation type <function softmax at 0x2b67d58c7ef0> does not have a native Nengo equivalent; falling back to a TensorNode
  "falling back to a TensorNode" % activation


In [3]:
conn = ndl_model.net.all_connections[3]
conn

<Connection at 0x2b67f54bb790 from <Neurons of <Ensemble "conv2d.0">> to <TensorNode "max_pooling2d">>

In [4]:
conn.post_obj.size_in

10816

In [5]:
get_isi_based_max_pooling_params("JKJK", ndl_model.model.layers)

In [11]:
NEURONS_LAST_SPIKED_TS[10816][15, 2:4, 2:4].flatten()

array([0., 0., 0., 0.])

In [2]:
def _get_conv_layer_output_shape(layer_name):
  for layer in ndl_model.model.layers:
    if layer.name == layer_name.split(".")[0]:
      return layer.output.shape[1:] # (num_chnls, rows, cols)

In [23]:
print(_get_conv_layer_output_shape("max_pooling2d_4"))

(16, 13, 13)


In [22]:
ndl_model.model.layers[2].name

'max_pooling2d_4'

In [13]:
5//2

2